In [ ]:
%matplotlib inline

from networkx.algorithms import connectivity as conn
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
from tools import utils
import networkx as nx
import time 
import os
import glob
import json

os.chdir("D:\Documenti\Lightning Network Plots")
graphs_list = [None] * len(glob.glob("*json"))
distance_list = [None] * len(glob.glob("*json"))
nodes_size_list = [None] * len(glob.glob("*json"))
edges_size_list = [None] * len(glob.glob("*json"))
avg_degree_list = [None] * len(glob.glob("*json"))
eccentricity_list = [None] * len(glob.glob("*json"))
k_connectivity_list = [None] * len(glob.glob("*json"))
betweenness_centrality_lists = [None] * len(glob.glob("*json"))
# Sets the picture size to a proper dimension.
plt.rcParams['figure.figsize'] = (10, 6)

# Create an array of snapshots of the network
for i, file in enumerate(glob.glob("*.json")):
    with open(file) as f:
        json_graph = json.load(f)
        graph = utils.parse_graph_from_json(json_graph)
        utils.clean_isolated_nodes(graph)
        graph = utils.fix_connectivity(graph)
        graphs_list[i] = graph
        nodes_size_list[i] = nx.number_of_nodes(graph)
        edges_size_list[i] = nx.number_of_edges(graph)
        degree_values = [x[1] for x in graph.degree()]
        sum_of_edges = sum(degree_values) / float(nx.number_of_nodes(graph))
        avg_degree_list[i] = sum_of_edges
        last_graph = graph


In [ ]:
for i, graph in enumerate(graphs_list):
    eccentricity_values = [x[1] for x in nx.eccentricity(graph).items()]
    max_eccentricity = max(eccentricity_values)
    min_eccentricity = min(eccentricity_values)
    eccentricity_list[i] = (max_eccentricity, min_eccentricity)

plt.plot(eccentricity_list)
plt.savefig('eccentricity.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(avg_degree_list)
plt.ylabel('Avg degree')
plt.savefig('avg_degree.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(nodes_size_list)
plt.ylabel('Number of nodes')
plt.savefig('number_of_nodes.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(edges_size_list)
plt.ylabel('Number of edges')
plt.savefig('number_of_edges.png', bbox_inches='tight') 
plt.show()

In [ ]:
for i, graph in enumerate(graphs_list):
    distance_list[i] = nx.diameter(graph)

plt.plot(distance_list)
plt.ylabel('Distance')
plt.savefig('distance.png', bbox_inches='tight')
plt.show()

Calculate the Betweenness Centrality measures for all the snapshot of the network. Returns the subgraph of those network with the most important nodes (BC value greater than 3 times the mean) along with the values returned by the betweenness centality.

Then plots the percentage of important nodes (and their respective edges) wrt the total number of nodes and edges.
**Average duration for 15 snapshots: 5 mins.**

In [ ]:
for i, graph in enumerate(graphs_list):
    betweenness_centrality_lists[i] = utils.important_nodes(graph)

nodes_ratio_betweenness_centrality = [None] * len(betweenness_centrality_lists)
edges_ratio_betweenness_centrality = [None] * len(betweenness_centrality_lists)

for i, elem in enumerate(betweenness_centrality_lists):
    nodes_ratio_betweenness_centrality[i] = elem[0].number_of_nodes() * 100.0 / nodes_size_list[i]
    edges_ratio_betweenness_centrality[i] = elem[0].number_of_edges() * 100.0 / edges_size_list[i]

plt.plot(nodes_ratio_betweenness_centrality, 'r--')
plt.plot(edges_ratio_betweenness_centrality, 'b')
plt.show()

Calculates k-vertex connectivity, this task is CPU intensive. Average of 5 mins per snapshot.

In [ ]:
new_path = r'D:\Documenti\Lightning Network Plots\k_conn'
if not os.path.exists(new_path):
    os.makedirs(new_path)
os.chdir(new_path)
start = time.time()
for i, graph in enumerate(graphs_list): 
    k_connectivity_list[i] = conn.k_components(graph)
    end = time.time()
    for key in k_connectivity_list[i].keys():
        for j in range(len(k_connectivity_list[i][key])):
            k_connectivity_list[i][key][j] = list(k_connectivity_list[i][key][j])
    with open('k_conn_' + str(i) + '.json', 'w') as f:
        json.dump(k_connectivity_list[i], f, indent=4)
    elapsed = end - start
    print('Task number ' + str(i) + ' finished in ' + str(elapsed) + ' seconds.')